In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset = pd.read_excel('/content/drive/MyDrive/Dataset.xlsx')
dataset.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,S1,k1,k2,k3,k4,k5,k6,k7,klass
0,1,1,1,0,1,1,0,1,1
1,1,1,1,0,1,1,1,1,1
2,1,1,1,0,1,1,0,1,1
3,1,1,1,0,1,1,0,1,1
4,1,1,1,0,1,1,0,1,1


In [ ]:
X = dataset.drop('klass', axis=1)
Y = dataset['klass']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

1.0
[[  89    0]
 [   0 1442]]


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

Замер времени

In [ ]:
import time
start_tree1 = time.perf_counter()
clf = DecisionTreeClassifier()
end_tree1 = time.perf_counter()

start_tree2 = time.perf_counter()
clf.fit(X_train, y_train)
end_tree2 = time.perf_counter()

start_tree3 = time.perf_counter()
y_pred1 = clf.predict(X_test)
end_tree3 = time.perf_counter()

start_forest1 = time.perf_counter()
clf = RandomForestClassifier(n_estimators=100, max_depth=9, max_features=2, min_samples_leaf=1, criterion='gini', bootstrap=True)
end_forest1 = time.perf_counter()

start_forest2 = time.perf_counter()
clf.fit(X_train, y_train)
end_forest2 = time.perf_counter()

start_forest3 = time.perf_counter()
y_pred2 = clf.predict(X_test)
end_forest3 = time.perf_counter()

print(f'Tree - Forest:\n{accuracy_score(y_test, y_pred1)} - {accuracy_score(y_test, y_pred2)}\n{end_tree1-start_tree1} - {end_forest1-start_forest1}\n{end_tree2-start_tree2} - {end_forest2-start_forest2}\n{end_tree3-start_tree3} - {end_forest3-start_forest3}')

Tree - Forest:
0.9991834068267189 - 0.9991834068267189
6.564800060004927e-05 - 0.00017638599820202217
0.0040787329999147914 - 0.17001155900288722
0.0029355270016822033 - 0.05320623400257318


Классификация

In [ ]:
# clf = RandomForestClassifier(n_estimators=100, max_depth=9, max_features=2, min_samples_leaf=1, criterion='gini', bootstrap=True)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# print('Model Accuracy:', accuracy_score(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))
clf = RandomForestClassifier(n_estimators=100)#, max_depth=9, max_features=2, min_samples_leaf=1, criterion='gini', bootstrap=True)
scores = cross_val_score(clf, X, Y, cv=5)
print(scores)
print("%0.6f accuracy with a standard deviation of %0.6f" % (scores.mean(), scores.std()))

[1.         0.99934683 1.         0.99934641 1.        ]
0.999739 accuracy with a standard deviation of 0.000320


Скачивание модели и загрузка

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

NotFittedError: ignored

Нахождение лучших параметров случайным

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

param = {'max_depth': range(3, 13), 
          'n_estimators':[100, 150, 200], #[50, 100, 150, 200, 250, 300], 
          'max_features': range(1,4),
          # 'criterion': ['gini', 'entropy'],
          # 'bootstrap':[True, False],
          'min_samples_leaf': range(1,4)}

rnd_search = RandomizedSearchCV(RandomForestClassifier(), param, n_jobs=-1, n_iter=10, cv=5)
rnd_search.fit(X, Y)
print(rnd_search.best_params_)
print(rnd_search.best_score_)

{'n_estimators': 150, 'min_samples_leaf': 1, 'max_features': 3, 'max_depth': 8}
0.9997386474729234


In [ ]:
df = pd.DataFrame(rnd_search.cv_results_)
df.to_excel('./RandomForest1.xlsx', index=False)

Нахождение лучших параметров по сетке

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(RandomForestClassifier(), param, n_jobs=-1, cv=5)
grid.fit(X, Y)
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 4, 'max_features': 2, 'min_samples_leaf': 1, 'n_estimators': 200}
1.0


In [ ]:
df = pd.DataFrame(grid.cv_results_)
df.to_excel('./RandomForest2.xlsx', index=False)